# CrateDB and Linear Regression

In this notebook, we import Twitter data from CrateDB as a Pandas dataframe and then fit a linear regression model on that data. The goal is to predict the number of followers a user has depending on the number of people they are following, using regression analysis.

In [ ]:
import numpy as np
import pandas as pd 

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from matplotlib import rcParams
plt.style.use('ggplot')
# you must manually install `crate` in the Anaconda environment, like so:
#
#     /anaconda3/bin/pip install crate

In [ ]:
rcParams['axes.grid'] = True
rcParams['figure.figsize'] = 10,5

## Load the Data From CrateDB

In [ ]:
limit = 100000
sql_query = """
    SELECT DISTINCT account_user['id'] AS id,
                    account_user['followers_count'] AS followers,
                    account_user['friends_count'] AS followees
               FROM tweets
              LIMIT {};
    """.format(limit)

# SQL query works out of the box with SQLAlchemy
try:
    # query CrateDB
    df_data = pd.read_sql(
        sql_query, 'crate://localhost:4200', index_col='id')
    # let's have a look at the first 5 rows
    display(df_data.head(5))
except Exception as error:
    print(error)
    print('CreatDB running and tweets imported?')

## Explore the Data

In [ ]:
df_data.plot.scatter(x='followees', y='followers')
plt.show()

## Preprocess Data

In [ ]:
# remove row if one value is zero, because log is -inf on zero
df_data = df_data.loc[~(df_data == 0).any(axis=1)] 

# apply log to followers and followees
df_data_transformed = df_data.apply(np.log10 )

# let's have a look at the first 5 rows after preprocessing data
display(df_data_transformed.head(5))

## Plot the Processed Data 

In [ ]:
ax = df_data_transformed.plot.scatter(x='followees', y='followers')
ax.set(xlabel='log(followees)', ylabel='log(followers)')

## Split Into Training and Testing Data

In [ ]:
# randomly split in train and test data
df_train, df_test = train_test_split(df_data_transformed, test_size=(1/3), random_state=42)

## Create a Base Model

In [ ]:
average_followers = df_train.followers.mean()
display('Average followers '+ str(average_followers))

## Evaluate the Base Model

In [ ]:
# always use average_followers
followers_pred = np.full(len(df_test.followers), average_followers)

# the root mean squared error in the log space
print("Root mean squared error: %.2f" 
    % np.sqrt(mean_squared_error(df_test.followers, followers_pred)))

# variance score: 1 is perfect prediction
print('Variance score: %.2f'
    % r2_score(df_test.followers, followers_pred))

## Create and Train a Linear Regression Model 

In [ ]:
# create linear regression object
regr = LinearRegression()

# train the model using the training set
regr.fit(df_train[['followees']], df_train.followers)

## Evaluate the Linear Regression Model 

In [ ]:
# make predictions using the testing set
followers_pred = regr.predict(df_test[['followees']])

# the root mean squared error in the log space
print("Root mean squared error: %.2f"
      % np.sqrt(mean_squared_error(df_test.followers, followers_pred)))

# explained variance score: 1 is perfect prediction
print('Variance score: %.2f'
      % r2_score(df_test.followers, followers_pred))

## Compare the Models

In [ ]:
# plot outputs
ax = df_test.plot.scatter(x='followees', y='followers')
ax.set(xlabel='log(followees)', ylabel='log(followers)')
plt.plot(df_test.followees, followers_pred, color='red')

## Get the Original Values

In [ ]:
# return it as a Dataframe
df_predicted = df_test[['followees']]
df_predicted['predicted_followers'] = followers_pred

# rescale to represent the actuall friends and follower count and return it as
# int not float 
df_predicted = df_predicted.apply(np.exp).astype(int)
df_predicted.head(5)

## Write Back to CrateDB

In [ ]:
# promote id from index type back as column, so it shows up in the table
df_predicted_reset = df_predicted.reset_index() 
df_predicted_reset.to_sql(
    'predicted_followers', 'crate://localhost',
    if_exists='append', index=False )